In [2]:
ls -R /aiffel/aiffel/jellyfish/

/aiffel/aiffel/jellyfish/:
barrel_jellyfish/   jellyfish-types.zip       Moon_jellyfish/
blue_jellyfish/     lions_mane_jellyfish/     Train_Test_Valid/
compass_jellyfish/  mauve_stinger_jellyfish/

/aiffel/aiffel/jellyfish/barrel_jellyfish:
01.jpeg  37.jpg          aug-18-24.jpg   aug-45-05.jpg  aug-72-38.jpg
02.jpg   38.jpg          aug-19-25.jpg   aug-46-06.jpg  aug-73-39.JPG
03.jpg   39.JPG          aug-20-27.jpg   aug-47-07.jpg  aug-74-40.jpg
04.jpg   40.jpg          aug-2-03.jpg    aug-48-10.jpg  aug-75-41.jpg
05.jpg   41.jpg          aug-21-28.jpg   aug-49-11.jpg  aug-76-42.jpg
06.jpg   42.jpg          aug-22-29.jpg   aug-50-12.jpg  aug-77-45.jpg
07.jpg   45.jpg          aug-23-30.jpg   aug-5-06.jpg   aug-78-46.jpg
10.jpg   46.jpg          aug-24-31.jpg   aug-51-13.jpg  aug-79-47.jpg
11.jpg   47.jpg          aug-25-32.JPG   aug-52-14.jpg  aug-80-48.jpg
12.jpg   48.jpg          aug-26-33.jpg   aug-53-15.jpg  aug-8-11.jpg
13.jpg   49.jpg          aug-27-34.jpg   aug-54-17.jpg  aug

In [12]:
data_dir = "/aiffel/aiffel/jellyfish/Train_Test_Valid"

print(os.path.exists(data_dir))  # ✅ True가 나와야 함
print(os.path.exists(os.path.join(data_dir, "train")))  # ✅ True가 나와야 함

True
True


# 데이터 전처리 밑 로드

In [13]:
import os
import torch
import torchvision.transforms as transforms
from torchvision import datasets
from torch.utils.data import DataLoader

# 데이터 경로 설정
data_dir = "/aiffel/aiffel/jellyfish/Train_Test_Valid"

# 데이터 변환 (Image Augmentation 포함)
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # 모델 입력 크기 맞춤
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

# 데이터 로드
train_dataset = datasets.ImageFolder(root=os.path.join(data_dir, 'train'), transform=transform)
valid_dataset = datasets.ImageFolder(root=os.path.join(data_dir, 'valid'), transform=transform)
test_dataset  = datasets.ImageFolder(root=os.path.join(data_dir, 'test'), transform=transform)

# 데이터로더 설정
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)
test_loader  = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# 클래스 확인
class_names = train_dataset.classes
print("클래스 목록:", class_names)


클래스 목록: ['Moon_jellyfish', 'barrel_jellyfish', 'blue_jellyfish', 'compass_jellyfish', 'lions_mane_jellyfish', 'mauve_stinger_jellyfish']


# 모델 정의 및 설정

In [14]:
import torch.nn as nn
import torchvision.models as models

# GPU 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 모델 선택 (Pretrained ResNet 사용)
model = models.resnet18(pretrained=True)

# 마지막 출력층 수정 (해파리 6개 클래스 분류)
num_classes = len(class_names)
model.fc = nn.Linear(model.fc.in_features, num_classes)

# 모델을 GPU로 이동
model = model.to(device)

# 손실 함수 및 최적화 함수 설정
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /aiffel/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

# 모델 학습

In [15]:
def train_model(model, train_loader, valid_loader, criterion, optimizer, num_epochs=10):
    for epoch in range(num_epochs):
        model.train()
        train_loss, train_acc = 0, 0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()

            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            train_acc += (outputs.argmax(1) == labels).sum().item()

        train_loss /= len(train_loader)
        train_acc /= len(train_loader.dataset)

        # 검증 단계
        model.eval()
        valid_loss, valid_acc = 0, 0

        with torch.no_grad():
            for images, labels in valid_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)

                valid_loss += loss.item()
                valid_acc += (outputs.argmax(1) == labels).sum().item()

        valid_loss /= len(valid_loader)
        valid_acc /= len(valid_loader.dataset)

        print(f"Epoch {epoch+1}/{num_epochs}: Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, Val Loss: {valid_loss:.4f}, Val Acc: {valid_acc:.4f}")

train_model(model, train_loader, valid_loader, criterion, optimizer, num_epochs=10)


Epoch 1/10: Train Loss: 0.9392, Train Acc: 0.7000, Val Loss: 9.4656, Val Acc: 0.3333
Epoch 2/10: Train Loss: 0.7760, Train Acc: 0.7667, Val Loss: 1.0307, Val Acc: 0.6923
Epoch 3/10: Train Loss: 0.3188, Train Acc: 0.8967, Val Loss: 1.9754, Val Acc: 0.6410
Epoch 4/10: Train Loss: 0.2774, Train Acc: 0.9322, Val Loss: 1.4625, Val Acc: 0.6410
Epoch 5/10: Train Loss: 0.3009, Train Acc: 0.8922, Val Loss: 3.5491, Val Acc: 0.5897
Epoch 6/10: Train Loss: 0.2596, Train Acc: 0.9144, Val Loss: 1.2469, Val Acc: 0.6923
Epoch 7/10: Train Loss: 0.3792, Train Acc: 0.8844, Val Loss: 1.9338, Val Acc: 0.5385
Epoch 8/10: Train Loss: 0.3823, Train Acc: 0.8689, Val Loss: 1.7692, Val Acc: 0.6410
Epoch 9/10: Train Loss: 0.2831, Train Acc: 0.9289, Val Loss: 1.4909, Val Acc: 0.6923
Epoch 10/10: Train Loss: 0.3461, Train Acc: 0.8778, Val Loss: 3.0873, Val Acc: 0.4872


# 모델 평가 및 테스트

In [16]:
def test_model(model, test_loader):
    model.eval()
    correct, total = 0, 0

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)

            predicted = outputs.argmax(1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

    print(f"Test Accuracy: {correct / total:.4f}")

test_model(model, test_loader)


Test Accuracy: 0.6750


# 개별 이미지 예측

In [17]:
def test_model(model, test_loader):
    model.eval()
    correct, total = 0, 0

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)

            predicted = outputs.argmax(1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

    print(f"Test Accuracy: {correct / total:.4f}")

test_model(model, test_loader)


Test Accuracy: 0.6500


In [ ]:
import torch

# 모델 훈련 코드
# model = 모델 훈련 코드

# 훈련이 완료된 후 모델 저장
torch.save(model.state_dict(), "jellyfish_classifier.pth")
